In [42]:
# for a basin, use a nearest basin within the same cluster as the parameter source (five fold)
import glob, os, sys, toml, pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import run_multiple_paramsets_Derecho

In [10]:
infile_basin_info = f"/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/CAMELS_level1_basin_info.csv"
inpath_moasmo = "/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO_bigrange"
itertar = 1
suffix = 'SSECVtransfer'
numruns = 10
cv_num = 5
df_basin_info = pd.read_csv(infile_basin_info)

In [8]:
# load cluster information
infile = "../camels_cluster/Manuela_Brunner_2020/flood_cluster_memberships_CAMELS.txt"
df_cluster = pd.read_csv(infile)
df_cluster = df_cluster.rename(
    columns={"Camels_IDs": "hru_id", "flood_cluster": "clusters"}
)
df_cluster2 = pd.DataFrame()

for id in df_basin_info["hru_id"].values:
    dfi = df_cluster.loc[df_cluster["hru_id"] == id]
    df_cluster2 = pd.concat([df_cluster2, dfi])

df_cluster2.sel_index = np.arange(len(df_cluster2))
df_cluster = df_cluster2
del df_cluster2
df_cluster["clusters"] = df_cluster["clusters"] - 1  # starting from 0

if np.any(df_basin_info["hru_id"].values - df_cluster['hru_id'].values != 0):
    print("Mistmatch between basins and clusters")
else:
    print("basins and clusters match")

basins and clusters match


/glade/derecho/scratch/guoqiang/tmp/ipykernel_45484/575832937.py:13: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_cluster2.sel_index = np.arange(len(df_cluster2))


In [39]:
# transfer
all_index = np.arange(len(df_info))
for i in range(cv_num):
    test_index = all_index[i::cv_num]
    train_index = np.setdiff1d(all_index, test_index)

    # get source basin
    for b in test_index:
        print('processing basin', b)
        cluster_b = df_cluster.iloc[b]["clusters"]
        lat_b = df_basin_info.iloc[b]['lat_cen']
        lon_b = df_basin_info.iloc[b]['lon_cen']

        indexb = (df_cluster["clusters"].values == cluster_b) & (df_cluster.index.isin(train_index))
        indexb[b] = False
        lat_s = df_basin_info[indexb]['lat_cen'].values
        lon_s = df_basin_info[indexb]['lon_cen'].values
        index_s = np.argmin( (lat_b - lat_s )**2 + (lon_b - lon_s)**2 )

        basin_source = df_basin_info.index[indexb][index_s]

        if cluster_b != df_cluster.iloc[basin_source]["clusters"]:
            print('error')

        print('dist**2', (lat_b - df_basin_info.iloc[basin_source]['lat_cen'] )**2 + (lon_b - df_basin_info.iloc[basin_source]['lon_cen'])**2  )
        
        # copy parameter files
        tar_folder = f'{inpath_moasmo}/level1_{b}_MOASMOcalib/param_sets_{suffix}/'
        tar_init_paramfile = f'{inpath_moasmo}/level1_{b}_MOASMOcalib/param_sets/paramset_iter0_trial0.pkl'
        df_param0 = pd.read_pickle(tar_init_paramfile)
        os.makedirs(tar_folder, exist_ok=True)
        for n in range(numruns):
            src_file = f'{inpath_moasmo}/level1_{basin_source}_MOASMOcalib/param_sets_normKGE/paramset_iter{itertar}_trial{n}.pkl'
            if n == 0:
                dfn = pd.read_pickle(src_file)
                if np.any(dfn['Parameter']!=df_param0['Parameter']):
                    sys.exit('Wrong source basin!')
            _ = os.system(f'cp {src_file} {tar_folder}')
    

processing basin 0
dist**2 1.6282224520000055
processing basin 5
dist**2 0.3418801345999945
processing basin 10
dist**2 0.50771380679999
processing basin 15
dist**2 0.1758117338000011
processing basin 20
dist**2 0.008458510599999407
processing basin 25
dist**2 0.15349951720000055
processing basin 30
dist**2 0.4114414816999977
processing basin 35
dist**2 0.03314768890000139
processing basin 40
dist**2 0.4114414816999977
processing basin 45
dist**2 0.06592655890000274
processing basin 50
dist**2 0.03491073940000063
processing basin 55
dist**2 0.060513836499998905
processing basin 60
dist**2 0.3127552745000012
processing basin 65
dist**2 0.03589496450000215
processing basin 70
dist**2 0.05624435089999641
processing basin 75
dist**2 0.02587444239999965
processing basin 80
dist**2 0.0431991954000013
processing basin 85
dist**2 0.02443776100000062
processing basin 90
dist**2 0.03709106409999864
processing basin 95
dist**2 0.033029871299998716
processing basin 100
dist**2 0.045157380499994654

In [43]:
path_CTSM_case_all = f'/glade/work/guoqiang/CTSM_CAMELS/Calib_HH_MOASMO_bigrange'
iter_end =  1
for tarbasin in range(627):
    config_file = f'{path_CTSM_case_all}/configuration/_level1-{tarbasin}_config_MOASMO.toml'
    config = toml.load(config_file)
    
    # inputs
    file_parameter_list = config['file_calib_param']
    path_CTSM_base = config['path_CTSM_case']
    path_script_MOASMO = config['path_script_MOASMO']
    path_CTSM_source = config['path_CTSM_source']
    ref_streamflow = config['file_Qobs']
    
    if 'add_flow_file' in config:
        add_flow_file = config['add_flow_file']
    else:
        add_flow_file = 'NA'
    
    script_singlerun = f'{path_script_MOASMO}/run_one_paramset_Derecho.py'
    script_clone = f'{path_CTSM_source}/cime/scripts/create_clone'

    if config['path_calib'] == 'NA':
        path_MOASMOcalib = f'{path_CTSM_base}_MOASMOcalib'
    else:
        path_MOASMOcalib = config['path_calib']
        
    # outputs
    path_paramset = f'{path_MOASMOcalib}/param_sets_{suffix}'
    path_submit = f'{path_MOASMOcalib}/run_model_{suffix}'
    path_archive = f'{path_MOASMOcalib}/ctsm_outputs_{suffix}'
        
    os.makedirs(path_MOASMOcalib, exist_ok=True) 
    
    # evaluation period
    RUN_STARTDATE = config['RUN_STARTDATE']
    ignore_month = config['ignore_month']
    STOP_OPTION = config['STOP_OPTION']
    STOP_N = config['STOP_N']
    
    if 'nonstandard_evaluation' in config:
        nonstandard_evaluation = config['nonstandard_evaluation']
    else:
        nonstandard_evaluation = 'NA'
    
    # HPC job settings
    job_mode = config['job_mode']
    job_CTSMiteration = config['job_CTSMiteration']
    # job_controlMOASMO = config['job_controlMOASMO'] # not needed here
    
    date_start = (pd.Timestamp(RUN_STARTDATE) + pd.offsets.DateOffset(months=ignore_month)).strftime('%Y-%m-%d') # ignor the first year when evaluating model
    if STOP_OPTION == 'nyears':
        date_end = (pd.Timestamp(RUN_STARTDATE) + pd.offsets.DateOffset(years=STOP_N)).strftime('%Y-%m-%d')
    elif STOP_OPTION == 'nmonths':
        date_end = (pd.Timestamp(RUN_STARTDATE) + pd.offsets.DateOffset(months=STOP_N)).strftime('%Y-%m-%d')
    else:
        sys.exit(f'STOP_OPTION must be nyears or nmonths. {STOP_OPTION} is not accepted.')

    # generate submission commands (note, this won't submit a real job on Derecho)
    run_multiple_paramsets_Derecho.generate_and_submit_multi_CTSM_runs(iter_end, path_submit, path_paramset, path_CTSM_base, 
                                                                       path_archive, script_singlerun, script_clone, 
                                                                       date_start, date_end, ref_streamflow, add_flow_file,
                                                                       job_CTSMiteration, job_mode)
